# [SciPy](https://scipy.github.io/devdocs/tutorial/index.html)
![SciPy](https://raw.githubusercontent.com/scipy/scipy-sphinx-theme/master/_static/scipyshiny_small.png)


# Spis Treści

1. [Stałe fizyczne](#physical_constants)
2. [Optymalizacja](#optimize)
    * [Dopasowanie MNK (Metoda Najmniejszych Kwadratów) używając curve_fit](#curve_fit)
        - [Ćwiczenie nr 1](#exercise_1)
        - [Niepewności i rozwiązanie początkowe](#uncertainties_guesses)
    * [Optymalizacja funkcji używając `minimize`](#minimize)
        - [Unbinned likelihood fits](#likelihood)
        - [Exercise 2](#exercise_2)
3. [Szybka Transformata Fouriera - Fast Fourier Transforms (FFTs)](#fft)
4. [Całkowanie](#integration)
    * [Całkowanie funkcji](#function_integration)
    * [Całkowanie próbki](#sample_integration)
        - [Ćwiczenie nr 3](#exercise_3)
5. [Interpolacja](#interpolation)
    * [Interpolacja liniowa](#linear_interpolation)
    * [Interpolacja wielomianami sklejanymi 3-go stopnia](#spline_interpolation)
    * [Ćwiczenie nr 4](#exercise_4)
6. [Statystyki](#stats)
    * [Rozkłady prawdopodobieństwa](#stats_distributions)
        - [Rozkłady ciągłe](#continuous_distributions)
        - [Rozkłady dyskretne](#discrete_distributions)
        - [Rozkłady wielowymiarowe](#multivariate_distributions)
    * [Funkcje statystyczne](#statistical_functions)
    * [Przykład](#stats_example)
    * [Ćwiczenie nr 5](#exercise_5)
7. [Funkcje specjalne](#special_functions)
    * [Funkcje Bessel-a](#bessel)
    * [Wielomiany ortogonalne](#ortho_polys)

# Importowanie i ustawienie notebook-a (uruchomić jako pierwsze!)

In [ ]:
import scipy as sp
import numpy as np

# we will need to plot stuff later
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2

<a id=physical_constants></a>
# Stałe fizyczne

In [ ]:
import scipy.constants as const
const.epsilon_0

In [ ]:
# zamiana jednostek temperatury:
const.convert_temperature(100, old_scale='C', new_scale='K')

In [ ]:
# więcej stałych (w tym jednostki i błędy)!
for k, v in const.physical_constants.items():
    print(k, ':', v)

In [ ]:
val, unit, uncertainty = const.physical_constants['muon mass energy equivalent in MeV']
val, unit, uncertainty

<a id=optimize></a>
# Optymalizacja

<a id=curve_fit></a>
## Dopasowanie MNK (Metoda Najmniejszych Kwadratów) używając `curve_fit`

Algorytm Levenberga-Marquardta do optymalizacji nieliniowej.

In [ ]:
a = -1
b = 5

def f(x, a, b):
    return np.exp(a * x) + b

x = np.linspace(0, 5, 100)
y = f(x, a, b) + np.random.normal(0, 0.1, 100)

plt.plot(x, y, '.', label='data')

In [ ]:
print(type(f))
f(1,2,2)

In [ ]:
from scipy.optimize import curve_fit

params, covariance_matrix = curve_fit(f, x, y)

uncertainties = np.sqrt(np.diag(covariance_matrix))
error = ((y - f(x, *params))**2).sum()

print('a = {:5.2f} ± {:.2f}'.format(params[0], uncertainties[0]))
print('b = {:5.2f} ± {:.2f}'.format(params[1], uncertainties[1]))
print('error: {}'.format(error))


In [ ]:
plt.plot(x, y, '.', label='data')
plt.plot(x, f(x, *params), label='fit result')
plt.legend();

In [ ]:
plt.plot(x,f(x, *params)-y,'.', label='residuals' )
plt.legend()

In [ ]:
# Regresja liniowa z curve_fit

def f_linear(x, a, b):
    return a*x + b

l_params, l_covariance_matrix = curve_fit(f_linear, x, y)
l_uncertainties = np.sqrt(np.diag(l_covariance_matrix))

plt.plot(x, y, '.', label='data')
plt.plot(x, f_linear(x, *l_params), label='fit result')
plt.legend();

error = ((y - f_linear(x, *l_params))**2).sum()
print('error: {}'.format(error))

In [ ]:
# Można sprawdzić, czy wynik jest taki sam jak ten uzyskany używając regresji liniowej
from scipy.stats import linregress

print('With curve_fit')
print('\ta = {:5.2f} ± {:.2f}'.format(l_params[0], l_uncertainties[0]))
print('\tb = {:5.2f} ± {:.2f}'.format(l_params[1], l_uncertainties[1]))

slope, intercept, r_value, p_value, std_err = linregress(x, y)
print('With linregress')
print('\ta = {:5.2f}'.format(slope))
print('\tb = {:5.2f}'.format(intercept))

In [ ]:
a = -1
b = 5
c = 5

def f(x, a, b):
    return np.exp(a * x) + b

def f_quadratic(x, a, b, c):
    return a*x**2 + b*x + c

x = np.linspace(0, 5, 100)
y = f(x, a, b) + np.random.normal(0, 0.1, 100)

l_params, l_covariance_matrix = curve_fit(f_quadratic, x, y)
l_uncertainties = np.sqrt(np.diag(l_covariance_matrix))

plt.plot(x, y, '.', label='data')
plt.plot(x, f_quadratic(x, *l_params), label='fit result')
plt.legend();

print('With curve_fit')
print('\ta = {:5.2f} ± {:.2f}'.format(l_params[0], l_uncertainties[0]))
print('\tb = {:5.2f} ± {:.2f}'.format(l_params[1], l_uncertainties[1]))
print('\tc = {:5.2f} ± {:.2f}'.format(l_params[2], l_uncertainties[2]))

error = ((y - f_quadratic(x, *l_params))**2).sum()
print('error: {}'.format(error))

<a id=exercise_1></a>
## Ćwiczenie nr 1

Użyj `curve_fit` do dopasowania wielomianu 4-tego stopnia do danych, narysuj wynik na wykresie i porównaj z poprzednimi dopasowaniami.

<a id=uncertainties_guesses></a>
### Niepewności i rozwiązanie początkowe

In [ ]:
x = np.linspace(0, 1, 1000)
def f(x, a, b):
    return np.sin(a * x + b)

y = f(x, 5 * np.pi, np.pi / 2) 
yerr = np.full_like(y, 0.2)
noise = np.random.normal(0, yerr, 1000)
y += noise


params, covariance_matrix = curve_fit(f, x, y)

# params, covariance_matrix = curve_fit(
#     f, x, y,
#     p0=[15, 2],
# )

# params, covariance_matrix = curve_fit(
#    f, x, y,
#    p0=[15, 1.5],
#    sigma=yerr,
#    absolute_sigma=True,
# )


# plot the stuff

x_plot = np.linspace(-0.1, 1.1, 1000)

plt.plot(x, y, '.', label='data')
plt.plot(x_plot, f(x_plot, *params), label='fit result')
plt.legend();

<a id=minimize></a>
## Optymalizacja funkcji używając `minimize`

Zdefiniujmy funkcję z dwoma lokalnymi minimami

In [ ]:
def f(x, h, x1):
    return h*np.exp(-((x - x1)**2).sum(axis=-1))

def g(x):
    return -f(x, 1, np.array([2, 1])) - f(x, 2, np.array([-2, 3]))

In [ ]:
XLIM = [-5, 5]
YLIM = [-2.5, 7.5]
x = np.linspace(-5, 5, 100)
y = np.linspace(-2.5, 7.5, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))
Z = g(pos)

In [ ]:
plt.contourf(X, Y, Z, cmap='viridis_r')
plt.colorbar()

In [ ]:
from scipy.optimize import minimize

result = minimize(g, x0=(0., 0.)) # rozwiązanie początkowe jest wymagane

print(result.x)

Nie znaleźliśmy globalnego minimum

In [ ]:
result = minimize(g, x0=(-2, 2))
print(result.x)

<a id=likelihood></a>
### Dopasowanie funkcji wiarygodności

Przykład: dopasowanie do ujemnej, zlogarytmowanej funkcji wiarygodności z rozkładu Poissona

In [ ]:
lambda_ = 15
k = np.random.poisson(lambda_, 100)

# trzeba użyć przedziałów wycentrowanych na liczbie całkowitej
bin_edges = np.arange(0, 31) - 0.5

plt.hist(k, bins=bin_edges);

Poisson PDF (Probability Density Function - Funkcja Gęstości Prawdopodobieństwa):

$$ 
f(k, \lambda) = \frac{\lambda^k}{k!} \mathrm{e}^{-\lambda}
$$

Funkcja wiarygodności:

$$
\mathcal{L} = \prod_{i=0}^{N} \frac{\lambda^{k_i}}{k_i!} \mathrm{e}^{-\lambda}
$$

Często łatwiej jest zoptymalizować logarytm $-\log(\mathcal{L})$, zobaczmy:

$$
-\log(\mathcal{L}) = - \sum_{i=0}^{N}\bigl( k_i \log(\lambda) - \log{k_i!} - \lambda \bigr)
$$

Interesuje nas minimum związane z $\lambda$, dlatego pomijamy stałe człony z nią niezwiązane
$$
-\log(\mathcal{L}) = \sum_{i=0}^{N}\bigl( \lambda - k_i \log(\lambda) \bigr)   
$$

Rzeczywiście łatwiej zoptymalizować taką funkcję.

In [ ]:
def negative_log_likelihood(lambda_, k):
    return np.sum(lambda_ - k * np.log(lambda_))

result = minimize(
    negative_log_likelihood,
    x0=(10, ),   # rozwiązanie początkowe
    args=(k, ),  # dodatkowe argumenty optymalizacji
)

result

print('True λ = {}'.format(lambda_))
print('Fit: λ = {:.2f} ± {:.2f}'.format(result.x[0], np.sqrt(result.hess_inv[0, 0])))

* `minimize` posiada sporo opcji dla różnych alogorytmów optymalizacji
* Dodatkowo może uwzględnić ogranicznie i warunki brzegowe (dla niektórych algorytmów)
* Warto zapisać swoją funkcję do optymalizacji i uprościć ją poprzez zlogarytmowanie

<a id=exercise_2></a>
### Ćwiczenie nr 2

Zrób to samo, żeby oszacować parametry rozkładu gaussa.

Wygeneruj próbkę z rozkładu normalnego z $\mu = 10$ oraz $\sigma = 6$

PDF:

$$
f(x, \mu, \sigma) =  \frac{1}{\sqrt{2 \pi}} \mathrm{e}^{-0.5 \frac{(x - \mu)^2}{\sigma^2}}
$$

Zoptymalizuj ujemną, zlogarytmowaną funkcję wiarygodności:

$$
-\log(\mathcal{L}) = -\sum_{i = 0}^N \log\bigl( \frac{1}{\sqrt{2 \pi}} \mathrm{e}^{-0.5 \frac{(x_i - \mu)^2}{\sigma^2}}  \bigr)
$$

Możesz użyć funkcji `norm.rvs` oraz `norm.pdf` do utworzenia próbki i zdefniowania funkcji do optymalizacji.

In [ ]:
from scipy.stats import norm

mi = 10
sig = 6
x = norm.rvs(mi,sig,1000)
y = norm.pdf(x,loc=mi,scale=sig)
# trzeba użyć przedziałów wycentrowanych na liczbie całkowitej
bin_edges = np.arange(-20, 40) - 0.5


plt.hist(x, bins=bin_edges)
# plt.figure()
# plt.plot(x,y)

def negative_log_likelihood(p, x):
    mi,sig= p
    return - np.sum(np.log(norm.pdf(x,loc=mi,scale=sig)))

result = minimize(
    negative_log_likelihood,
    x0=(mi,sig, ),   # rozwiązanie początkowe
    args=(x, ),  # dodatkowe argumenty do optymalizacji
    # bounds = [
    #   (None, None),
    #   (0.0, np.inf)
    # ],
)

result
xx = np.linspace(-20, 40, 1000)
plt.plot(xx, 1000*norm.pdf(xx,loc=result.x[0],scale=result.x[1]))

hess_inv= result.hess_inv
if not isinstance(hess_inv, np.ndarray):
    hess_inv= hess_inv.todense()

print('True mi = {} , sig = {}'.format(mi,sig))
print('Fit: mi = {:.2f} ± {:.2f}'.format(result.x[0], np.sqrt(hess_inv[0, 0])))
print('Fit: sig = {:.2f} ± {:.2f}'.format(result.x[1], np.sqrt(hess_inv[1, 1])))

<a id=fft></a>
# Szybka Transformata Fouriera - Fast Fourier Transforms (FFTs)

In [ ]:
freq1 = 4.9
freq2 = 7.9
N = 81001
n_harm = 15
t = np.linspace(1.2, 40.1, N)
# y = 6.1*np.sin(2*np.pi*freq1*t) + 4.2*np.sin(2*np.pi*freq2*t)

harms = np.linspace(1, n_harm, n_harm, endpoint=True)
y = np.sum([2**(n_harm-i+1)*np.sin(2*np.pi*freq1*i*t)/(2**(n_harm))*101 for i in harms], axis=0)

# dodaj szum
noisy_y = y + np.random.normal(np.full_like(y, 0.), 5)

In [ ]:
# %matplotlib inline
plt.scatter(t, noisy_y, s=10, alpha=0.25, lw=0)
plt.plot(t, y, c='k')
plt.xlim(1.2, 1.2+3*(1/freq1))
plt.xlabel(r'$t \ \ [\mathrm{d}]$')

In [ ]:
from scipy import fft

In [ ]:
krok = 1/(10*(t[-1]-t[0]))
nyq = 1/(2*(t[1]-t[0]))
size = int(nyq/krok)*2

print(N, nyq, krok, size)

In [ ]:
# %matplotlib inline

z = fft.fft(noisy_y, size)
f = fft.fftfreq(size, t[1] - t[0])

z = fft.fft(noisy_y, size)
f = fft.fftfreq(size, t[1] - t[0])

# plt.axvline(freq1, color='lightgray', lw=5)
# plt.axvline(freq2, color='lightgray', lw=5)
[plt.axvline(i*freq1, color='lightgray', lw=5) for i in harms]

# plt.xlim(0,20)
plt.xlim(0,(n_harm+1)*freq1)
plt.plot(f, 2/N*np.abs(z), lw=1.5) #np.abs(z)**2
plt.xlabel('frequency [$d^{-1}$]')
plt.ylabel('Amplitude [$mmag$]')
# plt.xscale('log')
# plt.yscale('log')

In [ ]:
# %matplotlib notebook
n_ph = 2
phases = (np.tile(t%(1/freq1)/(1/freq1), (n_ph,1)).T+np.arange(n_ph)).T

plt.plot(phases.flatten(), np.tile(noisy_y, n_ph), 'ob', ms=1)
# plt.plot(t%(1/freq2)/(1/freq2),noisy_y, 'or', ms=1)

<a id=integration></a>
# Całkowanie

O różnych metodach całkowania można poczytać w dokumentacji [Scipy documentation](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html).

<a id=function_integration></a>
## Całkowanie funkcji

`quad` używane do skończonych 1D całek numerycznych używając procedury napisanej w języku Fortran z biblioteki QUADPACK.

Przykład:
Zdefiniujmy funkcję do całkowania jako wielomian kwadratowy $f(x) = 3x^2 + 6x - 9$ na przedziale $x \in [0, 5]$. Analitycznie to:

$$ \int_0^5 3x^2 + 6x - 9 \ dx = \left[ x^3 + 3x^2 - 9x \right]_{x = 0}^{x = 5} = 155 $$

In [ ]:
from scipy.integrate import quad

def f(x):
    return 3*x**2 + 6*x - 9

quad(f, 0, 5)

Pierwszy zwracany parametr funkcji `quad` to wynik; drugi jest oszacowaniem błędu.

Dla 2D, 3D, oraz n-D całek , można użyć odpowiednio `dblquad`, `tplquad`, lub `nquad`.

Inne metody całkowania dostępne w Scipy:
* `quadrature` : [Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature)
* `romberg` : [Romberg integration](https://en.wikipedia.org/wiki/Romberg%27s_method)

Na przykład, weźmy funkcję $\mathrm{sinc}$:

$$
\mathrm{sinc}(x) \equiv
\begin{cases} 
1 & x = 0 \\
\sin(x)/x & \mathrm{otherwise}
\end{cases}
$$

In [ ]:
# %matplotlib notebook
x = np.linspace(-10, 10, 1000)
y = np.sinc(x)
plt.plot(x, y)
plt.title('Sinc Function')

res = quad(np.sinc, -10, 10)

plt.text(-10, 0.8, r'$ \int_{{-10}}^{{10}} \mathrm{{sinc}}(x) \ dx = {result:.4f}$ ?'.format(result=res[0]))

Również można to samo policzyć z Kwadratury Gaussa:

In [ ]:
from scipy.integrate import quadrature

print(quadrature(np.sinc, -10, 10)[0])

<a id=sampleintegration></a>
## Całkowanie próbki

Można użyć [funkcji interpolującej](#interpolation) i przekazać do funkcji `quad`. Lepszą alternatywą jest użycie specjalnych funkcji: `trapz`, `romb`, and `simps`.

Rozważmy znów funkcję $\mathrm{sinc}$. Najprostszą ale też dobrą metodą całkowania jest metoda trapezów: `trapz`:

In [ ]:
from scipy.integrate import trapz

# 50 punktów
x = np.linspace(-10, 10)
y = np.sinc(x)
print('  50 points:', trapz(y, x))   # najpierw y, potem x: y, x

# 1000 punktów
x = np.linspace(-10, 10, 1000)
y = np.sinc(x)
print('1000 points:', trapz(y, x))

<a id=exercise_3></a>
### Ćwiczenie nr 3

Zastosuj funkcję `trapz` do obliczenia całki:

$$
\int_{-4}^{4} \sqrt[3]{(1 - x^3)} dx
$$

**Wskazówka** użyj funkcji `np.cbrt`

In [ ]:
# %matplotlib notebook
from scipy.integrate import quadrature
from scipy.integrate import trapz

# f = lambda x: np.cbrt(1-np.power(x,3))

def f(x):
   return np.cbrt(1-np.power(x,3))

print(quadrature(f, -4, 4))

x = np.linspace(-4, 4,1000)
y = f(x)

print(trapz(y,x))
plt.plot(x,y, 'ok', ms=3, alpha=0.7)

### <a id=interpolation></a>
# Interpolacja

<a id=linear_interpolation></a>
## Interpolacja liniowa

Interpolację pomiędzy dwoma punktami $(x_0, y_0)$ i $(x_1, y_1)$ można wykonać wg. następującego wzoru:

$$y(x) = y_0 + (x - x_0) \frac{y_1 - y_0}{x_1 - x_0}$$

Wydaje się proste, ale uciążliwe. Dodatkowo, co w przypadku gdy chcemy aby argumenty mniejsze od $x_0$ miały wartość $y_0$ i analogicznie dla argumentów większych od $x_1$? Należy zastosować `if`-y i uwzględnić inne przypadki.

Zamiast tego, możemy skorzystać z prostej funkcji z modułu `scipy.interpolate` - `interp1d`.

In [ ]:
from scipy.interpolate import interp1d

x = (1, 2)
y = (5, 7)
print('Points:', list(zip(x, y)))

f = interp1d(x, y)
z = [1.25, 1.5, 1.75]
print('Interpolation:', list(zip(z, f(z))))

In [ ]:
plt.scatter(x, y)
plt.scatter(z, f(z))

W tej chwili, jeśli spróbujemy użyć wpółrzędnej x poza przedziałem $[x_0, x_1]$, podniesiony zostanie błąd `ValueError`:

In [ ]:
# f(2.5)   # odkomentuj

To dlatego, że nie wskazaliśmy `interp1d` jak ma sobie poradzić z granicami. Służy do tego argument `fill_value` i posiada kilka opcji:

1. Wartości poza przedziałem $[x_0, x_1]$ będą typu `float`.
2. Wartości $< x_0$ do `below` i wartości $> x_1$ do `above` przekazując krotkę, `(below, above)`.
3. Extrapolacja punktów poza przedział przekazując `extrapolate`.

Dodatkowo musimy powiedzieć `interp1d`, żeby nie podniósł błędu `ValueError` przypisując argumentowi `bounds_error` wartość `False`.

In [ ]:
z = [0.5, 1, 1.5, 2, 2.5]

f_a = interp1d(x, y, bounds_error=False, fill_value=0)   # wypełnij tę samą wartością z obu stron
print("Opcja nr 1:", list(zip(z, f(z))))

f_b = interp1d(x, y, bounds_error=False, fill_value=y)   # wypełnij wartością (below, above)
print("Opcja nr 2:", list(zip(z, f(z))))

f_c = interp1d(x, y, fill_value='extrapolate')   # ekstrapolacja, gdzie bounds_error automatycznie ma wartość False
print("Opcja nr 3:", list(zip(z, f(z))))

In [ ]:
plt.scatter(z, f_c(z))
plt.scatter(x, y)

<a id=spline_interpolation></a>
## Interpolacja wielomianem sklejanym 3-go stopnia

[Cubic splines](http://mathworld.wolfram.com/CubicSpline.html) najczęściej używane w przypadku gładkiej interpolacji pomiędzy punktami.

Jest na tyle popularne, że posiada własną implementację, `CubicSpline` - z reguły daje lepsze wyniki.

In [ ]:
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d

# Tworzymy próbkę sinus-ów
sample_x = np.linspace(-np.pi, np.pi, 5)
sample_y = np.sin(sample_x)

# rysujemy Sinus
z = np.linspace(np.min(sample_x), np.max(sample_x), 100)
plt.plot(z, np.sin(z), label='real')

# interpolacja 1D
f1 = interp1d(sample_x, sample_y)
plt.plot(z, f1(z), label='interp1d')

# wielomian sklejany 3-go stopnia
f2 = CubicSpline(sample_x, sample_y)
plt.plot(z, f2(z), label='CubicSpline')

plt.plot(sample_x, sample_y, 'ko')
plt.legend();
plt.grid(linestyle='--');

In [ ]:
plt.plot(z, f1(z)-np.sin(z), label='interp1d residuals', color='orange')
plt.plot(z, f2(z)-np.sin(z), label='CubicSpline residuals', color='green')
plt.legend();
plt.grid(linestyle='--');

Możliwa jes także interpolacja na dwu i więcej wymiarowych przestrzeniach (z pewnymi ograniczeniami).
Zobacz [dokumentację dot. interpolacji](https://docs.scipy.org/doc/scipy/reference/interpolate.html#module-scipy.interpolate)

<a id=exercise_4></a>
## Ćwiczenie nr 4

Wykonaj interpolację funkcji $sinc$:
* dwie próbki: 10 punktów i 100 punktów
* interpolacja liniowa i wielomianem sklejanym 3-go stopnia
* na przedziale [-10, 10]

<a id=stats></a>
# Statystyki

Moduł `scipy.stats` dostarcza głównie:
* rozkłady prawdopodobieństwa: ciągłe, dyskretne i wielowymiarowe
* funkcje statystyczne oraz testy

Więcej można poczytać w dokumentacji [dokumentacja modułu `stats`](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
from scipy import stats

<a id=stats_distributions></a>
## Rozkłady prawdopodobieństwa

Są trzy rodzaje rozkładów prawdopodobieństwa
* Continuous
* Discrete
* Multivariate

Każdy z nich ma pochodzi z tej samej klasy, więc mają te same metody.

<a id=continuous_distributions></a>
### Rozkłady ciągłe

Jest ~100 różnych ciągłych rozkładów prawdopodobieństwa. Niektóre metody, które można do nich zastosować to:
* `cdf`: Dystrybuanta (Cumulative Distribution Function)
* `pdf`: Funkcja gęstości prawdopodobieństwa (Probability Density Function)
* `rvs`: Losowa próbka danych (Random Variable Sample)
* `ppf`: Odwrotna dystrybuanta (Percent Point Function)
* `fit`: zwraca estymatory z MNW:  lokalizacja, skala i rozmiar

In [ ]:
std_normal = stats.norm()

x = np.linspace(-3, 3, 100)
plt.plot(x, std_normal.pdf(x))
plt.title('Standard Normal - Probability Density Function')
plt.xlabel('x')
plt.ylabel(r'$ f(x) =  \frac{1}{\sqrt{2 \pi}} \mathrm{e}^{-\frac{1}{2} x^2}}$')

In [ ]:
plt.plot(x, std_normal.cdf(x))
plt.title('Standard Normal - Cumulative Distribution Function')
plt.xlabel('x')
plt.ylabel(r'$ F(x) =  \frac{1}{\sqrt{2 \pi}} \int_{-\infty}^{x}\mathrm{e}^{-\frac{1}{2} x^2}}$')

In [ ]:
x_sample = std_normal.rvs(1000)
hist_result = plt.hist(x_sample, range=[-3, 3], bins=100, density=True)
x_plot = np.linspace(-3, 3, 100)
plt.plot(x_plot, std_normal.pdf(x_plot))
plt.title('Standard Normal - Random Sample')
plt.xlabel('x')

In [ ]:
# Estymatory średniej i ochylenia standardowego
stats.norm.fit(x_sample)

In [ ]:
N_SAMPLES = 1000

# Nazwa rozkładu, klasa, zakres wartości do narysowania
pds = [('Normal', stats.norm(), (-4., 4.)), 
      ('LogNormal', stats.lognorm(1.), (0., 4.)),
      ('Students T', stats.t(3.), (-10., 10.)),
      ('Chi Squared', stats.chi2(1.), (0., 10.))]

n_pds = len(pds)
fig, ax_list = plt.subplots(n_pds, 3, figsize=(5.*n_pds, 10.))
for ind, elem in enumerate(pds):
    
    pd_name, pd_func, pd_range = elem

    x_range = np.linspace(*pd_range, num=101)
    
    # Funkcja gęstości prawdopodobieństwa
    ax_list[ind, 0].plot(x_range, pd_func.pdf(x_range))
    ax_list[ind, 0].set_ylabel(pd_name)
    
    # Dystrybuanta
    ax_list[ind, 1].plot(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].fill_between(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].set_ylim([0., 1.])
    
    # losowa próbka
    ax_list[ind, 2].hist(pd_func.rvs(size=N_SAMPLES), bins=50)
    
    if ind == 0:
        _ = ax_list[ind, 0].set_title('Probability Density Function')
        _ = ax_list[ind, 1].set_title('Cumulative Distribution Function')
        _ = ax_list[ind, 2].set_title('Random Sample')

<a id=discrete_distributions></a>
### Rozkłady dyskretne
Prawie to samo, zamiast `pdf` jest `pmf`= Probability Mass Function

In [ ]:
N_SAMPLES = 1000

pds = [('Binomial', stats.binom(20, 0.7), (0., 21.)),
      ('Poisson', stats.poisson(10.), (0., 21.))]

n_pds = len(pds)
fig, ax_list = plt.subplots(n_pds, 3)
fig.set_size_inches((8.*n_pds, 8.))
for ind, elem in enumerate(pds):
    
    pd_name, pd_func, pd_range = elem

    x_range = np.arange(*pd_range)
    x_range_cdf = np.arange(*pd_range,0.1)

    # Funkcja masy prawdopodobieństwa
    ax_list[ind, 0].bar(x_range, pd_func.pmf(x_range))
    ax_list[ind, 0].set_ylabel(pd_name)
    
    # Dystrybuanta
    ax_list[ind, 1].plot(x_range_cdf, pd_func.cdf(x_range_cdf))
    ax_list[ind, 1].fill_between(x_range_cdf, pd_func.cdf(x_range_cdf))
    ax_list[ind, 1].set_ylim([0., 1.])
    
    # losowa próbka
    ax_list[ind, 2].hist(pd_func.rvs(size=N_SAMPLES), bins=x_range - 0.5)
    
    if ind == 0:
        _ = ax_list[ind, 0].set_title('Probability Mass Function')
        _ = ax_list[ind, 1].set_title('Cumulative Distribution Function')
        _ = ax_list[ind, 2].set_title('Random Sample')
        
plt.tight_layout()

<a id=multivariate_distributions></a>
### Rozkłady wielowymiarowe

In [ ]:
mult_mean = [0.1, 2.]
mult_cov =  [[2.0, 0.3], 
             [0.3, 0.5]]
mult_norm = stats.multivariate_normal(mean=mult_mean, cov=mult_cov)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors

XLIM = [-6., 6.]
YLIM = [0., 4.]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16., 8.))

# Make data.
x = np.linspace(*XLIM, num=100)
y = np.linspace(*YLIM, num=100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))
Z = mult_norm.pdf(pos)
S = mult_norm.rvs(1000)

color_norm = colors.Normalize(vmin=0, vmax=0.5)

# Contour plot of the PDF
ax1.contourf(X, Y, Z, cmap='coolwarm', norm=color_norm)
ax1.set_title('Probability Density Function')


# 2D histogram of the random sample
h = ax2.hist2d(S[:, 0], S[:, 1], bins=50, density=True, 
               cmap='coolwarm', range=[XLIM, YLIM],
              norm=color_norm)
ax2.set_title('Random sample')

# Add a common color bar
fig.colorbar(h[3], ax=[ax1, ax2]);

<a id=statistical_functions></a>
## Funkcje statystyczne

### Test normalności

Używany do sprawdzenia czy próbka pochodzi z rozkładu normalnego

In [ ]:
x_sample = stats.norm().rvs(size=1000)

alpha = 1e-3

In [ ]:
plt.hist(x_sample, bins=50);

`normaltest`: sprawdza czy próbka pochodzi z rozkładu normalnego

In [ ]:
def print_result(p, alpha):
    print("p = {:g}".format(p))
    if p < alpha:  # hipoteza zerowa H0: x pochodzi z rozkładu normalnego
        print("H0 odrzucona")
    else:
        print("H0 nie odrzucona")  

In [ ]:
k2, p = stats.normaltest(x_sample)
print_result(p, alpha)

### Kolmogorov-Smirnov test

Sprawdza czy próbka pochodzi z danego rozkładu.

In [ ]:
l = x_sample.mean()
s = x_sample.std(ddof=1)
print('loc={} scale={}'.format(l, s))

k, p = stats.kstest(x_sample, stats.norm(l, s).cdf)
print_result(p, alpha)


I wiele więcej...
Zobacz [dokumentację](https://docs.scipy.org/doc/scipy/reference/stats.html)

<a id=stats_example></a>
## Przykład

In [ ]:
import os
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
data_path = os.path.join("..","data","scipy_data","stock.csv")
df_prices = pd.read_csv(data_path)
df_prices.head(10)

In [ ]:
_ = df_prices[['Apple', 'Microsoft']].plot(title='2016 stock prices', figsize=(6., 6.))

In [ ]:
# Oblicz unormowane dzienne wzrosty
df_incs = df_prices.drop('Date', axis=1)
df_incs = ((df_incs - df_incs.shift(1))/df_incs.shift(1)).loc[1:, :]
df_incs['Date'] = df_prices.Date
df_incs.head(10)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12., 6.))
_ = df_incs[['Apple', 'Microsoft']].plot(ax=ax1)
_ = ax2.scatter(df_incs['Apple'], df_incs['Microsoft'])
ax2.grid()
ax2.set_xlabel('Apple')
ax2.set_ylabel('Microsoft')
plt.tight_layout()

Użyjemy metody `fit` aby uzyskać estymatory średniej i odchylenia standardowego.

In [ ]:
p = stats.norm.fit(df_incs.Apple)
print(p)

In [ ]:
# Tworzymy rozkłady na podstawie estymatorów
app_dist = stats.norm(*p)

In [ ]:
# Sprawdźmy czy próbka pochodzi z rozkładu normalnego (test Kolmogorowa-Smirnowa)
app_K, app_p = stats.kstest(df_incs['Apple'], app_dist.cdf)
print_result(app_p, alpha)

<a id=exercise_5></a>
## Ćwiczenie nr 5

Zrób to samo dla Microsoft-u:
* Dopasuj rozkład normalny
* Stwórz rozkład na podstawie dopasowania
* Sprawdź czy próbka pochodzi z rozkładu normalnego

<a id=special_functions></a>
# Funkcje specjalne

Pełna lista znajduje się [tutaj](https://docs.scipy.org/doc/scipy-0.14.0/reference/special.html).

<a id=bessel></a>
## Funkcje Bessel-a

In [ ]:
from scipy.special import jn

x = np.linspace(0, 10, 100)
for n in range(6):
    plt.plot(x, jn(n, x), label=r'$\mathtt{J}_{%i}(x)$' % n)
plt.grid()
plt.legend();

In [ ]:
import mpl_toolkits.mplot3d.axes3d as plt3d
from matplotlib.colors import LogNorm

def airy_disk(x):
    mask = x != 0
    result = np.empty_like(x)
    result[~mask] = 1.0
    result[mask] = (2 * jn(1, x[mask]) / x[mask])**2
    return result

# 2D plot
r = np.linspace(-10, 10, 500)
plt.plot(r, airy_disk(r))

# 3D plot
x = np.arange(-10, 10.1, 0.1)
y = np.arange(-10, 10.1, 0.1)

X, Y = np.meshgrid(x, y)
Z = airy_disk(np.sqrt(X**2 + Y**2))

result
fig = plt.figure()
ax = plt3d.Axes3D(fig)
ax.plot_surface(X, Y, Z, cmap='gray', norm=LogNorm(), lw=0)

None

<a id=erf></a>
## Funkcja błędu i dystrybuanta rozkładu Gaussa

$$\mathrm{erf}(z) = \frac{2}{\sqrt{\pi}} \int_0^z \exp\left( -t^2 \right) dt $$
$$\mathrm{ndtr}(z) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^z \exp\left( \frac{-t^2}{2} \right) dt $$

In [ ]:
from scipy.special import erf, ndtr

def gaussian(z):
    return np.exp(-z**2)

x = np.linspace(-3, 3, 100)
plt.plot(x, gaussian(x), label='Gaussian')
plt.plot(x, erf(x), label='Error Function')
plt.plot(x, ndtr(x), label='Gaussian CDF')
plt.ylim(-1.1, 1.1)
plt.legend(loc='best');

<a id=ortho_polys></a>
## Wielomiany ortogonalne

In [ ]:
from scipy.special import eval_legendre, eval_laguerre, eval_hermite, eval_chebyt

ortho_poly_dict = {'Legendre': eval_legendre,
                   'Laguerre': eval_laguerre,
                   'Hermite': eval_hermite,
                   'Chebyshev T': eval_chebyt}

def plot_ortho_poly(name):
    plt.figure()
    f = ortho_poly_dict[name]
    x = np.linspace(-1, 1, 100)
    for n in range(5):
        plt.plot(x, f(n, x), label='n = %i' % n)
    if name in ['Legendre', 'Chebyshev']:
        plt.ylim(-1.1, 1.1)
    plt.legend(loc='best', fontsize=16)
    plt.title(name + ' Polynomials')
    
plot_ortho_poly('Legendre')
# plot_ortho_poly('Laguerre')
# plot_ortho_poly('Hermite')
# plot_ortho_poly('Chebyshev T')

## Dodatkowe informacje

#### Dopasowywanie z  (i)minuit

Jednym z najbardziej rozbudowanych pakietów do optymalizacji/minimalizacji funkcji jest [MINUIT](https://en.wikipedia.org/wiki/MINUIT). Python pozwala używać kody napisanego w C++ za pomocą pakietu [iminuit](https://github.com/iminuit/iminuit).

Więcej informacji w [dokumentacji iminuit](http://iminuit.readthedocs.io/en/latest/), oraz w tutorialach [dostępne tutoriale](https://github.com/scikit-hep/iminuit/tree/develop/doc/tutorial) na Git-Hub.



